In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        # self.activation = nn.ReLU()
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, c2=0.9, line_search_method="bisect")
opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=0.1, c2=0.9, line_search_method="bisect")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0A

tensor(27.6712, grad_fn=<MseLossBackward0>) tensor(1.2048, grad_fn=<AddBackward0>) tensor(-60.6571) tensor(9.2885)
A

tensor(5.3298, grad_fn=<MseLossBackward0>) tensor(0.9468, grad_fn=<AddBackward0>) tensor(-27.9000) tensor(9.2885)
B

tensor(0.6210, grad_fn=<MseLossBackward0>) tensor(1.0758, grad_fn=<AddBackward0>) tensor(-9.5291) tensor(9.2885)
A

tensor(2.4014, grad_fn=<MseLossBackward0>) tensor(1.0113, grad_fn=<AddBackward0>) tensor(-18.8957) tensor(9.2885)
A

tensor(1.3649, grad_fn=<MseLossBackward0>) tensor(0.9791, grad_fn=<AddBackward0>) tensor(-14.2584) tensor(9.2885)
B

tensor(0.9560, grad_fn=<MseLossBackward0>) tensor(0.9952, grad_fn=<AddBackward0>) tensor(-11.9053) tensor(9.2885)
A

tensor(1.1512, grad_fn=<MseLossBackward0>) tensor(0.9871, grad_fn=<AddBackward0>) tensor(-13.0847) tensor(9.2885)
A

tensor(1.0513, grad_fn=<MseLossBackward0>) tensor(0.9831, grad_fn=<AddBackward0>) tensor(-12.4958) tensor(9.2885)
A

tensor(1.0031, grad_fn=<MseLossBackward0>) tensor(0.

KeyboardInterrupt: 

In [ ]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

ValueError: Input contains NaN.

In [ ]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, c2=0.5, tau=0.1, line_search_method="bisect")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  00.5
0.5
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.75
0.7

In [ ]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = -172471.54553764663
Test metrics:  R2 = -152449.0186521553
